# Criando a estrutura do Banco de Dados (MySQL)

In [ ]:
USE advocacia;

/*Criando as tabelas principais*/
CREATE TABLE IF NOT EXISTS pessoa (
    IdPessoa INT AUTO_INCREMENT,
    Nome VARCHAR(100) NOT NULL,    
    CONSTRAINT PK_Pessoa PRIMARY KEY (IdPessoa)
);

CREATE TABLE IF NOT EXISTS cliente (
    Id_Pessoa INT,    
    CONSTRAINT PK_Cliente PRIMARY KEY (Id_Pessoa),
    CONSTRAINT FK_Cliente_Pessoa FOREIGN KEY (Id_Pessoa) REFERENCES pessoa (IdPessoa) ON DELETE CASCADE
);

CREATE TABLE IF NOT EXISTS telefone (
    IdTelefone INT AUTO_INCREMENT,
    Ddi VARCHAR(4) NOT NULL DEFAULT '55',
    Ddd VARCHAR(4) NOT NULL DEFAULT '17',
    Telefone VARCHAR(15) NOT NULL,
    Whatsapp ENUM('S', 'N'),
    Privacidade ENUM('Particular', 'Comercial'),
    Tipo ENUM('Fixo', 'Celular'),
    CONSTRAINT PK_Telefone PRIMARY KEY (IdTelefone),
    CONSTRAINT UQ_Telefone__Ddd_Telefone UNIQUE (Ddd, Telefone)
);

CREATE TABLE IF NOT EXISTS pessoa_telefone (
    Id_Pessoa INT,
    Id_Telefone INT,
    CONSTRAINT PK_Pessoa_Telefone PRIMARY KEY (Id_Pessoa, Id_Telefone),
    CONSTRAINT FK_Pessoa_Telefone_Pessoa FOREIGN KEY (Id_Pessoa) REFERENCES pessoa (IdPessoa),
    CONSTRAINT FK_Pessoa_Telefone_Telefone FOREIGN KEY (Id_Telefone) REFERENCES telefone (IdTelefone)
);

CREATE TABLE IF NOT EXISTS email (
    IdEmail INT AUTO_INCREMENT,    
    Email VARCHAR(50) NOT NULL,
    Tipo ENUM('Primario', 'Secundario', 'Outro'),
    CONSTRAINT PK_Email PRIMARY KEY (IdEmail),
    CONSTRAINT UQ_Email__Email UNIQUE (Email),
    CONSTRAINT CK_Email__Email CHECK (Email REGEXP '^[A-Za-z0-9._%-]+@[A-Za-z0-9.-]+\\.[A-Z|a-z]{2,4}$')
);


CREATE TABLE IF NOT EXISTS pessoa_email (
    Id_Pessoa INT,
    Id_Email INT,
    CONSTRAINT PK_Pessoa_Email PRIMARY KEY (Id_Pessoa, Id_Email),
    CONSTRAINT FK_Pessoa_Email_Pessoa FOREIGN KEY (Id_Pessoa) REFERENCES pessoa (IdPessoa),
    CONSTRAINT FK_Pessoa_Email_Email FOREIGN KEY (Id_Email) REFERENCES email (IdEmail)
);

CREATE TABLE IF NOT EXISTS uf (
    IdUf INT AUTO_INCREMENT,
    Uf CHAR(2) NOT NULL,
    CONSTRAINT PK_Uf PRIMARY KEY (IdUf),
    CONSTRAINT UQ_Uf__Uf UNIQUE (Uf)
);

/*Inserindo todas as UF do Brasil na Tabela*/
INSERT IGNORE INTO uf (Uf) VALUES
    ('AC'), ('AL'), ('AM'), ('AP'), ('BA'), ('CE'), ('DF'), ('ES'), ('GO'),
    ('MA'), ('MG'), ('MS'), ('MT'), ('PA'), ('PB'), ('PE'), ('PI'), ('PR'),
    ('RJ'), ('RN'), ('RO'), ('RR'), ('RS'), ('SC'), ('SE'), ('SP'), ('TO');
    
CREATE TABLE IF NOT EXISTS advogado (
    Id_Pessoa INT,
    Oab VARCHAR(8) NOT NULL,
    Id_Uf INT NOT NULL DEFAULT 26,
    CONSTRAINT PK_Advogado PRIMARY KEY (Id_Pessoa),
    CONSTRAINT FK_Advogado_Pessoa FOREIGN KEY (Id_Pessoa) REFERENCES pessoa (IdPessoa) ON DELETE CASCADE,
    CONSTRAINT FK_Advogado_Uf FOREIGN KEY (Id_Uf) REFERENCES uf (IdUf),
    CONSTRAINT UQ_Advogado__Oab_IdUf UNIQUE (Oab, Id_Uf)
);

CREATE TABLE IF NOT EXISTS profissao (
    IdProfissao INT AUTO_INCREMENT,
    Profissao VARCHAR(100) NOT NULL,
    CONSTRAINT PK_Profissao PRIMARY KEY (IdProfissao),
    CONSTRAINT UQ_Profissao__Profissao UNIQUE (Profissao)
);

CREATE TABLE IF NOT EXISTS nacionalidade (
    IdNacionalidade INT AUTO_INCREMENT,
    Nacionalidade VARCHAR(100) NOT NULL,
    CONSTRAINT PK_Nacionalidade PRIMARY KEY (IdNacionalidade),
    CONSTRAINT UQ_Nacionalidade__Nacionalidade UNIQUE (Nacionalidade)
);

INSERT IGNORE INTO nacionalidade (Nacionalidade) VALUES ('Brasileiro(a)');

CREATE TABLE IF NOT EXISTS rg (
    IdRg INT AUTO_INCREMENT,
    Rg VARCHAR (15) NOT NULL,    
    Id_Uf INT NOT NULL DEFAULT 26,
    CONSTRAINT PK_Rg PRIMARY KEY (IdRg),
    CONSTRAINT FK_Rg_Uf FOREIGN KEY (Id_Uf) REFERENCES uf (IdUf),
    CONSTRAINT UQ_Rg__Rg_IdUf UNIQUE (Rg, Id_Uf)
);

CREATE TABLE IF NOT EXISTS pf (
    Id_Pessoa INT,
    Cpf CHAR (14),
    Sexo CHAR (1) NOT NULL,
    EstadoCivil ENUM('Desconhecido', 'Solteiro(a)', 'Casado(a)', 'Divorciado(a)', 'Viúvo(a)', 'Separado(a)') DEFAULT 'Desconhecido',
    DataDeNascimento DATE,
    Id_Rg INT,
    Id_Profissao INT,
    Id_Nacionalidade INT DEFAULT 1,
    CONSTRAINT PK_Pf PRIMARY KEY (Id_Pessoa),
    CONSTRAINT FK_Pf_Pessoa FOREIGN KEY (Id_Pessoa) REFERENCES pessoa (IdPessoa),
    CONSTRAINT FK_Pf_Rg FOREIGN KEY (Id_Pessoa) REFERENCES rg (IdRg),
    CONSTRAINT FK_Pf_Profissao FOREIGN KEY (Id_Pessoa) REFERENCES profissao (IdProfissao),
    CONSTRAINT FK_Pf_Nacionalidade FOREIGN KEY (Id_Pessoa) REFERENCES nacionalidade (IdNacionalidade),
    CONSTRAINT UQ_Pf__Cpf UNIQUE (Cpf),
    CONSTRAINT CK_Pf__Cpf CHECK (Cpf REGEXP '^[0-9]{3}\.[0-9]{3}\.[0-9]{3}-[0-9]{2}$')
);

CREATE TABLE IF NOT EXISTS pj (
    Id_Pessoa INT,
    Cnpj CHAR (18) NOT NULL,
    Id_PfAdministrador INT NOT NULL,
    CONSTRAINT PK_Pj PRIMARY KEY (Id_Pessoa),
    CONSTRAINT FK_Pj_Pessoa FOREIGN KEY (Id_Pessoa) REFERENCES pessoa (IdPessoa),
    CONSTRAINT FK_Pj_Pf FOREIGN KEY (Id_PfAdministrador) REFERENCES pf (Id_Pessoa),    
    CONSTRAINT UQ_Pj__Cnpj UNIQUE (Cnpj),
    CONSTRAINT CK_Pj__Cnpj CHECK (Cnpj REGEXP '^[0-9]{2}\.[0-9]{3}\.[0-9]{3}/[0-9]{4}-[0-9]{2}$')
);

CREATE TABLE IF NOT EXISTS endereco (
    IdEndereco INT AUTO_INCREMENT,
    Cep CHAR (8) NOT NULL,
    Numero VARCHAR (20) NOT NULL,
    Logradouro VARCHAR (100) NOT NULL,
    Bairro VARCHAR (100) NOT NULL,
    Cidade VARCHAR (100) NOT NULL,
    Id_Uf INT NOT NULL,    
    CONSTRAINT PK_Endereco PRIMARY KEY (IdEndereco),
    CONSTRAINT FK_Endereco_Uf FOREIGN KEY (Id_Uf) REFERENCES uf (IdUf),    
    CONSTRAINT UQ_Endereco__Cep_Id_Uf UNIQUE (Cep, Numero, Logradouro, Bairro, Cidade, Id_Uf)    
);

CREATE TABLE IF NOT EXISTS pessoa_endereco (
    Id_Pessoa INT,
    Id_Endereco INT,
    Complemento VARCHAR(100),
    CONSTRAINT PK_Pessoa_Endereco PRIMARY KEY (Id_Pessoa, Id_Endereco, Complemento),
    CONSTRAINT FK_Pessoa_Endereco_Pessoa FOREIGN KEY (Id_Pessoa) REFERENCES pessoa (IdPessoa),
    CONSTRAINT FK_Pessoa_Endereco_Endereco FOREIGN KEY (Id_Endereco) REFERENCES endereco (IdEndereco)
);

CREATE TABLE IF NOT EXISTS tag (
    IdTag INT AUTO_INCREMENT,
    Tag VARCHAR(100) NOT NULL,
    CONSTRAINT PK_Tag PRIMARY KEY (IdTag),
    CONSTRAINT UQ_Tag__Tag UNIQUE (Tag)
);

INSERT IGNORE INTO tag (Tag) VALUES ('Condomínio');

CREATE TABLE IF NOT EXISTS atributotag (
    IdAtributoTag INT AUTO_INCREMENT,
    Tabela VARCHAR(100) NOT NULL,
    Id_Atributo INT,
    Id_Tag INT,
    CONSTRAINT PK_Atributotag PRIMARY KEY (IdAtributoTag),
    CONSTRAINT FK_Atributotag_Tag FOREIGN KEY (Id_Tag) REFERENCES tag (IdTag),
    CONSTRAINT UQ_Atributotag__Tabela_IdTag UNIQUE (Tabela, Id_Atributo, Id_Tag)
);

__________________
# Criando **VIEWS**



In [ ]:

/*Criando as Views principais*/
CREATE OR REPLACE VIEW vw_endereco AS
SELECT
    pe.IdPessoa,
    pe.Nome,    
    CONCAT(
        e.Logradouro,
        ', ',
        e.Numero,
        IFNULL(CONCAT(', ', pe_en.Complemento), ''),
        ', ',
        e.Bairro,
        ', ',
        e.Cidade,
        '/ ',
        uf.Uf,
        ' - ',
        e.Cep
    ) AS EnderecoCompleto
FROM pessoa pe
JOIN pessoa_endereco pe_en ON pe.IdPessoa = pe_en.Id_Pessoa
JOIN endereco e ON pe_en.Id_Endereco = e.IdEndereco
JOIN uf ON e.Id_Uf = uf.IdUf;


-- View para exibir todos os atributos de uma Pessoa Física (PF)
CREATE VIEW View_PF AS
SELECT
    pf.Id_Pessoa AS IdPessoa_PF,
    pf.Cpf,
    pf.Sexo,
    pf.EstadoCivil,
    pf.DataDeNascimento,
    rg.Rg,
    uf.Uf AS Uf_Rg,
    profissao.Profissao AS Profissao,
    nacionalidade.Nacionalidade AS Nacionalidade,
    email.Email,
    email.Tipo AS TipoEmail,
    telefone.Ddi,
    telefone.Ddd,
    telefone.Telefone,
    telefone.Whatsapp,
    telefone.Privacidade AS PrivacidadeTelefone,
    telefone.Tipo AS TipoTelefone,
    ve.EnderecoCompleto AS Endereco,
    t.Tag,
    CASE
        WHEN c.Id_Pessoa IS NOT NULL THEN 'Cliente'
        ELSE NULL
    END AS Cliente
FROM pf
LEFT JOIN rg ON pf.Id_Rg = rg.IdRg
LEFT JOIN uf ON rg.Id_Uf = uf.IdUf
LEFT JOIN profissao ON pf.Id_Profissao = profissao.IdProfissao
LEFT JOIN nacionalidade ON pf.Id_Nacionalidade = nacionalidade.IdNacionalidade
LEFT JOIN pessoa_email ON pf.Id_Pessoa = pessoa_email.Id_Pessoa
LEFT JOIN email ON pessoa_email.Id_Email = email.IdEmail
LEFT JOIN pessoa_telefone ON pf.Id_Pessoa = pessoa_telefone.Id_Pessoa
LEFT JOIN telefone ON pessoa_telefone.Id_Telefone = telefone.IdTelefone
LEFT JOIN vw_endereco ve ON ve.IdPessoa = pf.Id_Pessoa
LEFT JOIN atributotag att ON pf.Id_Pessoa = att.Id_Atributo AND att.Tabela = 'pessoa'
LEFT JOIN tag t ON att.Id_Tag = t.IdTag
LEFT JOIN cliente c ON pf.Id_Pessoa = c.Id_Pessoa;

 
-- View para exibir todos os atributos de uma Pessoa Jurídica (PJ)
CREATE VIEW View_PJ AS
SELECT
    pj.Id_Pessoa AS IdPessoa_PJ,
    pj.Cnpj,
    pessoa_administrador.Nome AS Administrador,
    email.Email,
    email.Tipo AS TipoEmail,
    telefone.Ddi,
    telefone.Ddd,
    telefone.Telefone,
    telefone.Whatsapp,
    telefone.Privacidade AS PrivacidadeTelefone,
    telefone.Tipo AS TipoTelefone,
    ve.EnderecoCompleto AS Endereco,
    t.Tag,
    CASE
        WHEN c.Id_Pessoa IS NOT NULL THEN 'Cliente'
        ELSE NULL
    END AS Cliente
FROM pj
LEFT JOIN pf pf_administrador ON pj.Id_PfAdministrador = pf_administrador.Id_Pessoa
LEFT JOIN pessoa pessoa_administrador ON pf_administrador.Id_Pessoa = pessoa_administrador.IdPessoa
LEFT JOIN pessoa_email ON pj.Id_Pessoa = pessoa_email.Id_Pessoa
LEFT JOIN email ON pessoa_email.Id_Email = email.IdEmail
LEFT JOIN pessoa_telefone ON pj.Id_Pessoa = pessoa_telefone.Id_Pessoa
LEFT JOIN telefone ON pessoa_telefone.Id_Telefone = telefone.IdTelefone
LEFT JOIN vw_endereco ve ON ve.IdPessoa = pj.Id_Pessoa
LEFT JOIN atributotag att ON pj.Id_Pessoa = att.Id_Atributo AND att.Tabela = 'pessoa'
LEFT JOIN tag t ON att.Id_Tag = t.IdTag
LEFT JOIN cliente c ON pj.Id_Pessoa = c.Id_Pessoa;
    
CREATE OR REPLACE VIEW vw_dados_pessoa AS
SELECT
    p.IdPessoa,
    p.Nome,
    COALESCE(pf.Cpf, pj.Cnpj) AS 'CPF/CNPJ',
    ve.EnderecoCompleto,
    CASE
        WHEN c.Id_Pessoa IS NOT NULL THEN 'Cliente'
        ELSE NULL
    END AS Cliente,
    t.Tag
FROM pessoa p
LEFT JOIN pf ON p.IdPessoa = pf.Id_Pessoa
LEFT JOIN pj ON p.IdPessoa = pj.Id_Pessoa
LEFT JOIN cliente c ON p.IdPessoa = c.Id_Pessoa
LEFT JOIN atributotag att ON p.IdPessoa = att.Id_Atributo AND att.Tabela = 'pessoa'
LEFT JOIN tag t ON att.Id_Tag = t.IdTag
LEFT JOIN vw_endereco ve ON ve.IdPessoa = p.IdPessoa;

__________
# Criação de **Procedures** para inserir dados no BD

In [ ]:
DELIMITER //

CREATE PROCEDURE InserirPessoa(
    IN p_nome VARCHAR(100)
)
BEGIN    
    INSERT INTO pessoa (Nome) VALUES (p_nome);    
END //

CREATE PROCEDURE InserirCliente(
    IN p_id_pessoa INT
)
BEGIN
    IF NOT EXISTS (SELECT 1 FROM cliente WHERE Id_Pessoa = p_id_pessoa) THEN
        INSERT INTO cliente (Id_Pessoa) VALUES (p_id_pessoa);
    END IF;
END //

CREATE PROCEDURE InserirAdvogado(
    IN p_id_pessoa INT,
    IN p_oab VARCHAR(8),
    IN p_oabuf CHAR(2)
)
BEGIN
    DECLARE v_id_uf INT;
    SET v_id_uf = (
        SELECT IdUf 
        FROM uf
        WHERE Uf = p_oabuf
    );

    IF NOT EXISTS (SELECT 1 FROM advogado WHERE Id_Pessoa = p_id_pessoa) THEN
        INSERT INTO advogado (Id_Pessoa, Oab, Id_Uf) VALUES (p_id_pessoa, p_oab, v_id_uf);
    END IF;
END //

CREATE PROCEDURE InserirProfissao(
    IN p_profissao VARCHAR(100)
)
BEGIN
    IF NOT EXISTS (SELECT 1 FROM profissao WHERE Profissao = p_profissao) THEN
        INSERT INTO profissao (Profissao) VALUES (p_profissao);
    END IF;
END //

CREATE PROCEDURE InserirNacionalidade(
    IN p_nacionalidade VARCHAR(100)
)
BEGIN
    IF NOT EXISTS (SELECT 1 FROM nacionalidade WHERE Nacionalidade = p_nacionalidade) THEN
        INSERT INTO nacionalidade (Nacionalidade) VALUES (p_nacionalidade);
    END IF;
END //

CREATE PROCEDURE InserirRg(
    IN p_rg VARCHAR(15),
    IN p_rguf CHAR(2)
)
BEGIN
    DECLARE v_id_uf INT;
    SET v_id_uf = (
        SELECT IdUf 
        FROM uf
        WHERE Uf = p_rguf
    );

    IF NOT EXISTS (SELECT 1 FROM rg WHERE Rg = p_rg AND Id_Uf = v_id_uf) THEN
        INSERT INTO rg (Rg, Id_Uf) VALUES (p_rg, v_id_uf);
    END IF;
END //

CREATE PROCEDURE InserirTelefone(
    IN p_ddi VARCHAR(4),
    IN p_ddd VARCHAR(4),
    IN p_telefone VARCHAR(15),
    IN p_whatsapp ENUM('S', 'N'),
    IN p_privacidade ENUM('Particular', 'Comercial'),
    IN p_tipo ENUM('Fixo', 'Celular')
)
BEGIN
    IF NOT EXISTS (SELECT 1 FROM telefone WHERE Ddd = p_ddd AND Telefone = p_telefone) THEN
        INSERT INTO telefone (Ddi, Ddd, Telefone, Whatsapp, Privacidade, Tipo) 
        VALUES (p_ddi, p_ddd, p_telefone, p_whatsapp, p_privacidade, p_tipo);
    END IF;
END //

CREATE PROCEDURE InserirPessoaTelefone(
    IN p_id_pessoa INT, 
    IN p_id_telefone INT    
)
BEGIN                
    IF NOT EXISTS (
        SELECT 1
        FROM pessoa_telefone pt
        WHERE pt.Id_Pessoa = p_id_pessoa
        AND pt.Id_Telefone = p_id_telefone        
    ) THEN
        INSERT INTO pessoa_telefone (Id_Pessoa, Id_Telefone)
        VALUES (p_id_pessoa, p_id_telefone);            
    END IF;
END //

CREATE PROCEDURE InserirEmail(
    IN p_email VARCHAR(50),
    IN p_tipo ENUM('Primario', 'Secundario', 'Outro')
)
BEGIN
    IF NOT EXISTS (SELECT 1 FROM email WHERE Email = p_email) THEN
        INSERT INTO email (Email, Tipo) VALUES (p_email, p_tipo);
    END IF;
END //

CREATE PROCEDURE InserirPessoaEmail(
    IN p_id_pessoa INT, 
    IN p_id_email INT    
)
BEGIN                
    IF NOT EXISTS (
        SELECT 1
        FROM pessoa_email pe
        WHERE pe.Id_Pessoa = p_id_pessoa
        AND pe.Id_Email = p_id_email        
    ) THEN
        INSERT INTO pessoa_email (Id_Pessoa, Id_Email)
        VALUES (p_id_pessoa, p_id_email);            
    END IF;
END //

CREATE PROCEDURE InserirEndereco(
    IN p_cep CHAR(8),
    IN p_numero VARCHAR(20),
    IN p_logradouro VARCHAR(100),
    IN p_complemento VARCHAR(100),
    IN p_bairro VARCHAR(100),
    IN p_cidade VARCHAR(100),
    IN p_enduf CHAR(2)
)
BEGIN
    DECLARE v_id_uf INT;
    SET v_id_uf = (
        SELECT IdUf 
        FROM uf
        WHERE Uf = p_enduf
    );    

    IF NOT EXISTS (
        SELECT 1
        FROM endereco e
        WHERE e.Cep = p_cep
        AND e.Numero = p_numero
        AND e.Logradouro = p_logradouro        
        AND e.Bairro = p_bairro
        AND e.Cidade = p_cidade
        AND e.Id_Uf = v_id_uf
    ) THEN
        INSERT INTO endereco (Cep, Numero, Logradouro, Bairro, Cidade, Id_Uf)
        VALUES (p_cep, p_numero, p_logradouro, p_bairro, p_cidade, v_id_uf);            
    END IF;
END //


CREATE PROCEDURE InserirPessoaEndereco(
    IN p_id_pessoa INT, 
    IN p_id_endereco INT, 
    IN p_complemento VARCHAR(100)    
)
BEGIN                
    IF NOT EXISTS (
        SELECT 1
        FROM pessoa_endereco pe
        WHERE pe.Id_Pessoa = p_id_pessoa
        AND pe.Id_Endereco = p_id_endereco
        AND pe.Complemento = p_complemento
    ) THEN
        INSERT INTO pessoa_endereco (Id_Pessoa, Id_Endereco, Complemento)
        VALUES (p_id_pessoa, p_id_endereco, p_complemento);            
    END IF;
END //

CREATE PROCEDURE InserirTag(
    IN p_tag VARCHAR(100)
)
BEGIN
    IF NOT EXISTS (SELECT 1 FROM tag WHERE Tag = p_tag) THEN
        INSERT INTO tag (Tag) VALUES (p_tag);
    END IF;
END //

CREATE PROCEDURE InserirAtributoTag(
    IN p_tabela VARCHAR(100), 
    IN p_id_atributo INT, 
    IN p_id_tag INT    
)
BEGIN                
    IF NOT EXISTS (
        SELECT 1
        FROM atributotag att
        WHERE att.Tabela = p_tabela
        AND att.Id_Atributo = p_id_atributo
        AND att.Id_Tag = p_id_tag
    ) THEN
        INSERT INTO atributotag (Tabela, Id_Atributo, Id_Tag)
        VALUES (p_tabela, p_id_atributo, p_id_tag);            
    END IF;
END //

CREATE PROCEDURE InserirPf(
    IN p_id_pessoa INT,
    IN p_cpf CHAR(14),
    IN p_sexo CHAR(1),
    IN p_estado_civil ENUM('Desconhecido', 'Solteiro(a)', 'Casado(a)', 'Divorciado(a)', 'Viúvo(a)', 'Separado(a)'),
    IN p_data_nascimento DATE,
    IN p_id_rg INT,
    IN p_id_profissao INT,
    IN p_id_nacionalidade INT
)
BEGIN
    IF NOT EXISTS (SELECT 1 FROM pf WHERE Cpf = p_cpf) THEN
        INSERT INTO pf (Id_Pessoa, Cpf, Sexo, EstadoCivil, DataDeNascimento, Id_Rg, Id_Profissao, Id_Nacionalidade)
        VALUES (
            p_id_pessoa,p_cpf, p_sexo, p_estado_civil, p_data_nascimento, p_id_rg, p_id_profissao, p_id_nacionalidade
        );
    END IF;
END //

CREATE PROCEDURE InserirPj(
    IN p_id_pessoa INT,
    IN p_cnpj CHAR(18),
    IN p_id_pf_administrador INT
)
BEGIN
    IF NOT EXISTS (SELECT 1 FROM pj WHERE Cnpj = p_cnpj) THEN
        INSERT INTO pj (Id_Pessoa, Cnpj, Id_PfAdministrador)
        VALUES (p_id_pessoa,p_cnpj, p_id_pf_administrador);
    END IF;
END //

DELIMITER ;

### Procedures mais complexas para PF e PJ


In [ ]:
DELIMITER //

CREATE PROCEDURE InserirDadosPf(
    IN p_nome VARCHAR(100),
    IN p_cpf CHAR(14),
    IN p_sexo CHAR(1),
    IN p_estado_civil ENUM('Desconhecido', 'Solteiro(a)', 'Casado(a)', 'Divorciado(a)', 'Viúvo(a)', 'Separado(a)'),
    IN p_data_nascimento DATE,
    IN p_rg VARCHAR(15),
    IN p_rguf INT,
    IN p_profissao INT,
    IN p_nacionalidade VARCHAR(100),
    IN p_ddi_tel VARCHAR(4),
    IN p_ddd_tel VARCHAR(4),
    IN p_telefone VARCHAR(15),
    IN p_whatsapp ENUM('S', 'N'),
    IN p_privacidade_tel ENUM('Particular', 'Comercial'),
    IN p_tipo_tel ENUM('Fixo', 'Celular'),
    IN p_email VARCHAR(50),
    IN p_tipo_email ENUM('Primario', 'Secundario', 'Outro'),
    IN p_cep_end CHAR(8),
    IN p_numero_end VARCHAR(20),
    IN p_logradouro_end VARCHAR(100),
    IN p_complemento_end VARCHAR(100),
    IN p_bairro_end VARCHAR(100),
    IN p_cidade_end VARCHAR(100),
    IN p_uf_end CHAR(2)
)
BEGIN
    DECLARE v_id_pessoa INT;
    DECLARE v_id_telefone INT;
    DECLARE v_id_email INT;
    DECLARE v_id_endereco INT;
    DECLARE v_id_rg INT;
    DECLARE v_id_profissao INT;
    DECLARE v_id_nacionalidade INT;
        
    -- Utilize DECLARE CONTINUE HANDLER FOR SQLEXCEPTION para lidar com exceções SQL
    DECLARE CONTINUE HANDLER FOR SQLEXCEPTION
    BEGIN
        ROLLBACK;
        SIGNAL SQLSTATE '45000'
            SET MESSAGE_TEXT = 'Erro durante a inserção dos dados de Pessoa Física. Operação cancelada. Detalhes: ' || SQLERRM();
    END;

    -- Inicia a transação
    START TRANSACTION;

    CALL InserirRg(p_rg, p_rguf);
    SET v_id_rg = (
        SELECT IdRg 
        FROM rg 
        WHERE Rg = p_rg AND Id_Uf = (
            SELECT IdUf FROM uf WHERE Uf = p_rguf)
    );

    CALL InserirProfissao(p_profissao);
    SET v_id_profissao = (
        SELECT IdProfissao 
        FROM profissao 
        WHERE Profissao = p_profissao
    );

    CALL InserirNacionalidade(p_nacionalidade);
    SET v_id_nacionalidade = (
        SELECT IdNacionalidade 
        FROM nacionalidade 
        WHERE Nacionalidade = p_nacionalidade
    );

    -- Inserir Pessoa
    CALL InserirPessoa(p_nome);
    SET v_id_pessoa = LAST_INSERT_ID();

    -- Inserir PF
    CALL InserirPf(v_id_pessoa, p_cpf, p_sexo, p_estado_civil, p_data_nascimento, v_id_rg, v_id_profissao, v_id_nacionalidade);

    -- Inserir Telefone
    CALL InserirTelefone(p_ddi_tel, p_ddd_tel, p_telefone, p_whatsapp, p_privacidade_tel, p_tipo_tel);
    SET v_id_telefone = LAST_INSERT_ID();

    -- Relacionar Pessoa e Telefone
    CALL InserirPessoaTelefone(v_id_pessoa, v_id_telefone);

    -- Inserir Email
    CALL InserirEmail(p_email, p_tipo_email);
    SET v_id_email = LAST_INSERT_ID();

    -- Relacionar Pessoa e Email
    CALL InserirPessoaEmail(v_id_pessoa, v_id_email);

    -- Inserir Endereço
    CALL InserirEndereco(p_cep_end, p_numero_end, p_logradouro_end, p_complemento_end, p_bairro_end, p_cidade_end, p_uf_end);
    SET v_id_endereco = LAST_INSERT_ID();

    -- Relacionar Pessoa e Endereço
    CALL InserirPessoaEndereco(v_id_pessoa, v_id_endereco, p_complemento_end);

    -- Efetiva a transação
    COMMIT;

END //

CREATE PROCEDURE InserirDadosPj(
    IN p_nome VARCHAR(100),
    IN p_cnpj CHAR(18),
    IN p_id_pf_administrador INT,
    IN p_ddi_tel VARCHAR(4),
    IN p_ddd_tel VARCHAR(4),
    IN p_telefone VARCHAR(15),
    IN p_whatsapp ENUM('S', 'N'),
    IN p_privacidade_tel ENUM('Particular', 'Comercial'),
    IN p_tipo_tel ENUM('Fixo', 'Celular'),
    IN p_email VARCHAR(50),
    IN p_tipo_email ENUM('Primario', 'Secundario', 'Outro'),
    IN p_cep_end CHAR(8),
    IN p_numero_end VARCHAR(20),
    IN p_logradouro_end VARCHAR(100),
    IN p_complemento_end VARCHAR(100),
    IN p_bairro_end VARCHAR(100),
    IN p_cidade_end VARCHAR(100),
    IN p_uf_end char(2)
)
BEGIN
    DECLARE v_id_pessoa INT;
    DECLARE v_id_telefone INT;
    DECLARE v_id_email INT;
    DECLARE v_id_endereco INT;
    
    
    -- Utilize DECLARE CONTINUE HANDLER FOR SQLEXCEPTION para lidar com exceções SQL
    DECLARE CONTINUE HANDLER FOR SQLEXCEPTION
    BEGIN
        ROLLBACK;
        SIGNAL SQLSTATE '45000'
            SET MESSAGE_TEXT = 'Erro durante a inserção dos dados de Pessoa Jurídica. Operação cancelada. Detalhes: ' || SQLERRM();
    END;

    -- Inicia a transação
    START TRANSACTION;

    -- Inserir Pessoa
    CALL InserirPessoa(p_nome);
    SET v_id_pessoa = LAST_INSERT_ID();

    -- Inserir PJ
    CALL InserirPj(v_id_pessoa, p_cnpj, p_id_pf_administrador);

   -- Inserir Telefone
    CALL InserirTelefone(p_ddi_tel, p_ddd_tel, p_telefone, p_whatsapp, p_privacidade_tel, p_tipo_tel);
    SET v_id_telefone = LAST_INSERT_ID();

    -- Relacionar Pessoa e Telefone
    CALL InserirPessoaTelefone(v_id_pessoa, v_id_telefone);

    -- Inserir Email
    CALL InserirEmail(p_email, p_tipo_email);
    SET v_id_email = LAST_INSERT_ID();

    -- Relacionar Pessoa e Email
    CALL InserirPessoaEmail(v_id_pessoa, v_id_email);

    -- Inserir Endereço
    CALL InserirEndereco(p_cep_end, p_numero_end, p_logradouro_end, p_complemento_end, p_bairro_end, p_cidade_end, p_uf_end);
    SET v_id_endereco = LAST_INSERT_ID();

    -- Relacionar Pessoa e Endereço
    CALL InserirPessoaEndereco(v_id_pessoa, v_id_endereco, p_complemento_end);

    -- Efetiva a transação
    COMMIT;

END //

DELIMITER ;

__________
# Criação de **Procedures** para *alterar* dados no BD (em andamento)

In [ ]:
DELIMITER //

CREATE PROCEDURE AtualizarPessoa(
    IN p_id_pessoa INT,
    IN p_nome VARCHAR(100)
)
BEGIN
    UPDATE pessoa SET Nome = p_nome WHERE IdPessoa = p_id_pessoa;
END //

CREATE PROCEDURE AtualizarAdvogado(
    IN p_id_pessoa INT,
    IN p_oab VARCHAR(8),
    IN p_oabuf CHAR(2)
)
BEGIN
    DECLARE v_id_uf INT;
    SET v_id_uf = (
        SELECT IdUf 
        FROM uf
        WHERE Uf = p_oabuf
    );
    UPDATE advogado SET Oab = p_oab, Id_Uf = v_id_uf  WHERE IdPessoa = p_id_pessoa;
END //

CREATE PROCEDURE AtualizarProfissao(
    IN p_idprofissao INT,
    IN p_profissao VARCHAR(100)
)
BEGIN
    UPDATE profissao SET Profissao = p_profissao WHERE IdProfissao = p_idprofissao;
END //

CREATE PROCEDURE AtualizarNacionalidade(
    IN p_idnacionalidade INT,
    IN p_nacionalidade VARCHAR(100)
)
BEGIN
    UPDATE nacionalidade SET Nacionalidade = p_nacionalidade WHERE IdNacionalidade = p_idnacionalidade;
END //

CREATE PROCEDURE AtualizarRg(
    IN p_idrg INT,
    IN p_rg VARCHAR(15),
    IN p_rguf CHAR(2)
)
BEGIN
    DECLARE v_id_uf INT;
    SET v_id_uf = (
        SELECT IdUf 
        FROM uf
        WHERE Uf = p_rguf
    );
    UPDATE rg SET Rg = p_rg, Id_Uf = v_id_uf  WHERE IdRg = p_idrg;
END //

CREATE PROCEDURE AtualizarTelefone(
    IN p_idtelefone INT,
    IN p_ddi VARCHAR(4),
    IN p_ddd VARCHAR(4),
    IN p_telefone VARCHAR(15),
    IN p_whatsapp ENUM('S', 'N'),
    IN p_privacidade ENUM('Particular', 'Comercial'),
    IN p_tipo ENUM('Fixo', 'Celular')
)
BEGIN
   UPDATE telefone 
   SET Ddi, Rg = p_rg, Id_Uf = v_id_uf  
   WHERE IdTelefone = p_idtelefone;
END //

CREATE PROCEDURE InserirPessoaTelefone(
    IN p_id_pessoa INT, 
    IN p_id_telefone INT    
)
BEGIN                
    IF NOT EXISTS (
        SELECT 1
        FROM pessoa_telefone pt
        WHERE pt.Id_Pessoa = p_id_pessoa
        AND pt.Id_Telefone = p_id_telefone        
    ) THEN
        INSERT INTO pessoa_telefone (Id_Pessoa, Id_Telefone)
        VALUES (p_id_pessoa, p_id_telefone);            
    END IF;
END //

CREATE PROCEDURE InserirEmail(
    IN p_email VARCHAR(50),
    IN p_tipo ENUM('Primario', 'Secundario', 'Outro')
)
BEGIN
    IF NOT EXISTS (SELECT 1 FROM email WHERE Email = p_email) THEN
        INSERT INTO email (Email, Tipo) VALUES (p_email, p_tipo);
    END IF;
END //

CREATE PROCEDURE InserirPessoaEmail(
    IN p_id_pessoa INT, 
    IN p_id_email INT    
)
BEGIN                
    IF NOT EXISTS (
        SELECT 1
        FROM pessoa_email pe
        WHERE pe.Id_Pessoa = p_id_pessoa
        AND pe.Id_Email = p_id_email        
    ) THEN
        INSERT INTO pessoa_email (Id_Pessoa, Id_Email)
        VALUES (p_id_pessoa, p_id_email);            
    END IF;
END //

CREATE PROCEDURE InserirEndereco(
    IN p_cep CHAR(8),
    IN p_numero VARCHAR(20),
    IN p_logradouro VARCHAR(100),
    IN p_complemento VARCHAR(100),
    IN p_bairro VARCHAR(100),
    IN p_cidade VARCHAR(100),
    IN p_enduf CHAR(2)
)
BEGIN
    DECLARE v_id_uf INT;
    SET v_id_uf = (
        SELECT IdUf 
        FROM uf
        WHERE Uf = p_enduf
    );    

    IF NOT EXISTS (
        SELECT 1
        FROM endereco e
        WHERE e.Cep = p_cep
        AND e.Numero = p_numero
        AND e.Logradouro = p_logradouro        
        AND e.Bairro = p_bairro
        AND e.Cidade = p_cidade
        AND e.Id_Uf = v_id_uf
    ) THEN
        INSERT INTO endereco (Cep, Numero, Logradouro, Bairro, Cidade, Id_Uf)
        VALUES (p_cep, p_numero, p_logradouro, p_bairro, p_cidade, v_id_uf);            
    END IF;
END //


CREATE PROCEDURE InserirPessoaEndereco(
    IN p_id_pessoa INT, 
    IN p_id_endereco INT, 
    IN p_complemento VARCHAR(100)    
)
BEGIN                
    IF NOT EXISTS (
        SELECT 1
        FROM pessoa_endereco pe
        WHERE pe.Id_Pessoa = p_id_pessoa
        AND pe.Id_Endereco = p_id_endereco
        AND pe.Complemento = p_complemento
    ) THEN
        INSERT INTO pessoa_endereco (Id_Pessoa, Id_Endereco, Complemento)
        VALUES (p_id_pessoa, p_id_endereco, p_complemento);            
    END IF;
END //

CREATE PROCEDURE InserirTag(
    IN p_tag VARCHAR(100)
)
BEGIN
    IF NOT EXISTS (SELECT 1 FROM tag WHERE Tag = p_tag) THEN
        INSERT INTO tag (Tag) VALUES (p_tag);
    END IF;
END //

CREATE PROCEDURE InserirAtributoTag(
    IN p_tabela VARCHAR(100), 
    IN p_id_atributo INT, 
    IN p_id_tag INT    
)
BEGIN                
    IF NOT EXISTS (
        SELECT 1
        FROM atributotag att
        WHERE att.Tabela = p_tabela
        AND att.Id_Atributo = p_id_atributo
        AND att.Id_Tag = p_id_tag
    ) THEN
        INSERT INTO atributotag (Tabela, Id_Atributo, Id_Tag)
        VALUES (p_tabela, p_id_atributo, p_id_tag);            
    END IF;
END //

CREATE PROCEDURE InserirPf(
    IN p_id_pessoa INT,
    IN p_cpf CHAR(14),
    IN p_sexo CHAR(1),
    IN p_estado_civil ENUM('Desconhecido', 'Solteiro(a)', 'Casado(a)', 'Divorciado(a)', 'Viúvo(a)', 'Separado(a)'),
    IN p_data_nascimento DATE,
    IN p_id_rg INT,
    IN p_id_profissao INT,
    IN p_id_nacionalidade INT
)
BEGIN
    IF NOT EXISTS (SELECT 1 FROM pf WHERE Cpf = p_cpf) THEN
        INSERT INTO pf (Id_Pessoa, Cpf, Sexo, EstadoCivil, DataDeNascimento, Id_Rg, Id_Profissao, Id_Nacionalidade)
        VALUES (
            p_id_pessoa,p_cpf, p_sexo, p_estado_civil, p_data_nascimento, p_id_rg, p_id_profissao, p_id_nacionalidade
        );
    END IF;
END //

CREATE PROCEDURE InserirPj(
    IN p_id_pessoa INT,
    IN p_cnpj CHAR(18),
    IN p_id_pf_administrador INT
)
BEGIN
    IF NOT EXISTS (SELECT 1 FROM pj WHERE Cnpj = p_cnpj) THEN
        INSERT INTO pj (Id_Pessoa, Cnpj, Id_PfAdministrador)
        VALUES (p_id_pessoa,p_cnpj, p_id_pf_administrador);
    END IF;
END //

DELIMITER ;